In [1]:
%pip install requests pandas tqdm python-dotenv

import os
from dotenv import load_dotenv
import requests
import pandas as pd
from tqdm import tqdm
import time

# Load .env from project root (adjust path if needed)
load_dotenv(r"c:\Users\DELL\Downloads\Revops Code\RevOps_Analytics_portfolio\.env")

# --- CONFIG ---
API_KEY = os.getenv("GOOGLE_API_KEY")  # loaded from .env
if not API_KEY:
    raise RuntimeError("GOOGLE_API_KEY not found in environment. Add it to the .env file.")

MAX_RESULTS = 150 # Set the maximum number of results you want to retrieve

# --- INPUTS ---
keyword = input("Enter business type (e.g. 'dentist', 'restaurant', 'marketing agency'): ")
location = input("Enter location (e.g. 'Ikeja Lagos', 'Abuja', 'Port Harcourt'): ")
radius = 10000  # meters

# --- PHASE 1: BASIC PLACE SEARCH WITH PAGINATION ---
search_url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query={keyword}+in+{location}&radius={radius}&key={API_KEY}"
all_places = []
next_page_token = None

while len(all_places) < MAX_RESULTS:
    if next_page_token:
        search_url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?pagetoken={next_page_token}&key={API_KEY}"
    elif all_places: # Only request first page once
      break # Exit loop if no next_page_token and not the first page


    response = requests.get(search_url)
    data = response.json()

    places = data.get("results", [])
    all_places.extend(places)
    print(f"✅ Found {len(places)} results in this page. Total results: {len(all_places)}")

    next_page_token = data.get("next_page_token")

    if not next_page_token or len(places) == 0:
        break
    time.sleep(2) # Add a delay to avoid rate limiting

print(f"✅ Finished collecting initial results. Total results: {len(all_places)}")

# --- PHASE 2: PLACE DETAILS ENRICHMENT ---
details_url = "https://maps.googleapis.com/maps/api/place/details/json"
enriched_data = []

# Limit the number of places to process based on MAX_RESULTS
places_to_process = all_places[:MAX_RESULTS]


for place in tqdm(places_to_process, desc="Enriching leads"):
    place_id = place.get("place_id")
    params = {
        "place_id": place_id,
        "fields": "name,formatted_address,formatted_phone_number,website,rating,opening_hours,geometry",
        "key": API_KEY
    }
    details_resp = requests.get(details_url, params=params).json()
    details = details_resp.get("result", {})

    enriched_data.append({
        "Name": details.get("name"),
        "Address": details.get("formatted_address"),
        "Phone": details.get("formatted_phone_number"),
        "Website": details.get("website"),
        "Rating": details.get("rating"),
        "Open_Now": details.get("opening_hours", {}).get("open_now") if details.get("opening_hours") else None,
        "Maps_URL": f"https://www.google.com/maps/place/?q=place_id:{place_id}",
        "Location": location # Add the location to the DataFrame
    })

df = pd.DataFrame(enriched_data)

# --- EXPORT ---
file_name = f"{keyword}_{location}_enriched_leads.csv".replace(" ", "_")
df.to_csv(file_name, index=False)

print(f"💾 Exported {len(df)} enriched leads to '{file_name}'")


Note: you may need to restart the kernel to use updated packages.
✅ Found 20 results in this page. Total results: 20
✅ Found 20 results in this page. Total results: 20
✅ Found 20 results in this page. Total results: 40
✅ Found 20 results in this page. Total results: 40
✅ Found 20 results in this page. Total results: 60
✅ Finished collecting initial results. Total results: 60
✅ Found 20 results in this page. Total results: 60
✅ Finished collecting initial results. Total results: 60


Enriching leads: 100%|██████████| 60/60 [00:36<00:00,  1.64it/s]

💾 Exported 60 enriched leads to 'supermarket_yaba_enriched_leads.csv'


In [2]:
import requests
import time
import pandas as pd
from datetime import datetime
import os
from dotenv import load_dotenv

# Load .env (if notebook restarted independently)
load_dotenv(r"c:\Users\DELL\Downloads\Revops Code\RevOps_Analytics_portfolio\.env")

# === HUBSPOT CONFIGURATION ===
HUBSPOT_API_KEY = os.getenv("HUBSPOT_API_KEY")  # loaded from .env
if not HUBSPOT_API_KEY:
    raise RuntimeError("HUBSPOT_API_KEY not found in environment. Add it to the .env file.")
BASE_URL = "https://api.hubapi.com"
COMPANY_URL = f"{BASE_URL}/crm/v3/objects/companies"

headers = {
    "Authorization": f"Bearer {HUBSPOT_API_KEY}",
    "Content-Type": "application/json"
}

# === FUNCTION: CHECK IF COMPANY EXISTS ===
def company_exists(domain=None, name=None):
    """Checks HubSpot for existing company by domain or name."""
    search_url = f"{BASE_URL}/crm/v3/objects/companies/search"
    filters = []

    if domain:
        filters.append({"propertyName": "domain", "operator": "EQ", "value": domain})
    if name and not domain:
        filters.append({"propertyName": "name", "operator": "EQ", "value": name})

    if not filters:
        return False

    payload = {"filterGroups": [{"filters": filters}]}
    resp = requests.post(search_url, headers=headers, json=payload)

    if resp.status_code != 200:
        print(f"⚠️ Search error for {name}: {resp.text}")
        return False

    results = resp.json().get("results", [])
    return len(results) > 0


# === FUNCTION: CREATE COMPANY ===
def create_company(row):
    """Creates company in HubSpot if not already existing."""
    if company_exists(domain=row["Website"], name=row["Name"]):
        return {"Name": row["Name"], "Status": "Duplicate", "Timestamp": datetime.now()}

    data = {
        "properties": {
            "name": row["Name"],
            "address": row["Address"],
            "phone": row["Phone"] or "",
            "domain": row["Website"] if row["Website"] else "",
            "city": row["Location"],
            # Removed the 'source' property as it caused an error
        }
    }

    response = requests.post(COMPANY_URL, headers=headers, json=data)

    if response.status_code == 201:
        print(f"✅ Added company: {row['Name']}")
        return {"Name": row["Name"], "Status": "Added", "Timestamp": datetime.now()}
    else:
        print(f"❌ Failed for {row['Name']}: {response.text}")
        return {"Name": row["Name"], "Status": "Failed", "Timestamp": datetime.now()}


# === LOOP THROUGH LEADS AND PUSH TO HUBSPOT ===
sync_log = []

for i, row in df.iterrows():
    result = create_company(row)
    sync_log.append(result)
    time.sleep(0.3)  # Prevent API rate limit issues

# === EXPORT SYNC LOG ===
log_df = pd.DataFrame(sync_log)
log_file_name = f"company_sync_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
log_df.to_csv(log_file_name, index=False)

print(f"📊 Sync Log saved as {log_file_name}")

✅ Added company: Mallmart Supermarket
✅ Added company: HANDY BASKET SUPERSTORE
✅ Added company: HANDY BASKET SUPERSTORE
✅ Added company: LAIT STORES
✅ Added company: LAIT STORES
✅ Added company: Tonyson Pharmacy and Stores
✅ Added company: Tonyson Pharmacy and Stores
✅ Added company: D-ARENA SUPERMARKET YABA
✅ Added company: D-ARENA SUPERMARKET YABA
✅ Added company: QMART SUPERMARKET
✅ Added company: QMART SUPERMARKET
✅ Added company: Melody Supermarket
✅ Added company: Melody Supermarket
✅ Added company: Lifeback Pharmacy and Store Limited
✅ Added company: Lifeback Pharmacy and Store Limited
✅ Added company: Addide Iwaya
✅ Added company: Addide Iwaya
✅ Added company: Staplestores
✅ Added company: Staplestores
✅ Added company: Yem-Yem Superstores LUTH
✅ Added company: Yem-Yem Superstores LUTH
✅ Added company: Bokku! Mart Herbert Macaulay way, Yaba
✅ Added company: Bokku! Mart Herbert Macaulay way, Yaba
✅ Added company: Megaland Supermarket
✅ Added company: Megaland Supermarket
✅ Added 

In [4]:
print(response.status_code, response.text)


200 {
   "html_attributions" : [],
   "results" : 
   [
      {
         "business_status" : "OPERATIONAL",
         "formatted_address" : "21 Oshodi Rd, Mafoluku Oshodi, Lagos 102214, Lagos, Nigeria",
         "geometry" : 
         {
            "location" : 
            {
               "lat" : 6.556146099999999,
               "lng" : 3.3489553
            },
            "viewport" : 
            {
               "northeast" : 
               {
                  "lat" : 6.55757587989272,
                  "lng" : 3.350309379892722
               },
               "southwest" : 
               {
                  "lat" : 6.554876220107277,
                  "lng" : 3.347609720107278
               }
            }
         },
         "icon" : "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/shopping-71.png",
         "icon_background_color" : "#4B96F3",
         "icon_mask_base_uri" : "https://maps.gstatic.com/mapfiles/place_api/icons/v2/shoppingcart_pinlet",
         "n